<a href="https://colab.research.google.com/github/CP200435/-_TEAM_9124/blob/main/%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A_TEAM_9124.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.243 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.5/112.6 GB disk)


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
#下載資料集
import gdown
import os
import shutil

#下載training_image.zip
# gdown.download("https://drive.google.com/uc?export=download&id=1FfrsOgeJKSHtRM3Q649HJvSF1l4blxpl","/content/training_image.zip")
gdown.download("https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6","/content/training_image.zip")

#下載training_label.zip
# gdown.download("https://drive.google.com/uc?export=download&id=1KmYS2JhAexpWWlzlo_qwoX1TifA1COWW","/content/training_label.zip")
gdown.download("https://drive.google.com/uc?export=download&id=1K4n7G8zVK0_PtgkkAQFBEXJs-zJpElKG","/content/training_label.zip")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6
From (redirected): https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6&confirm=t&uuid=89fbf056-e4d1-4b15-92d4-17355c356c15
To: /content/training_image.zip
100%|██████████| 1.83G/1.83G [00:21<00:00, 84.0MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1K4n7G8zVK0_PtgkkAQFBEXJs-zJpElKG
To: /content/training_label.zip
100%|██████████| 669k/669k [00:00<00:00, 128MB/s]


'/content/training_label.zip'

In [ ]:
from textwrap import dedent
from pathlib import Path

data_yaml = dedent(
    """
train: "./datasets/train/images"
val: "./datasets/val/images"
test: "./datasets/test/images"

names:
  0: aortic_valve
"""
).strip()

config_path = Path("aortic_valve_colab.yaml")
config_path.write_text(data_yaml + "\n", encoding="utf-8")

print(f"YAML saved to: {config_path.resolve()}")
print(data_yaml)


YAML saved to: /content/aortic_valve_colab.yaml
train: "./datasets/train/images"
val: "./datasets/val/images"
test: "./datasets/test/images"

names:
  0: aortic_valve


In [ ]:
#下載資料集
import gdown
import os
!mkdir ./datasets
!mkdir ./datasets/test
gdown.download("https://drive.google.com/uc?export=download&id=13y81TV-z4_fbm7fCmjb0w_FvnPEZIHHa","/content/testing.zip")
# !unzip '/content/datasets/testing' -d '/content/datasets/test/tmp'

Downloading...
From (original): https://drive.google.com/uc?export=download&id=13y81TV-z4_fbm7fCmjb0w_FvnPEZIHHa
From (redirected): https://drive.google.com/uc?export=download&id=13y81TV-z4_fbm7fCmjb0w_FvnPEZIHHa&confirm=t&uuid=404d2e1b-f1ab-4a6a-80ea-34553aa7f3cf
To: /content/testing.zip
100%|██████████| 1.83G/1.83G [00:16<00:00, 112MB/s]


'/content/testing.zip'

In [ ]:
import os, zipfile, shutil  #week10hw

def unzip_if_needed(zip_path, dest_dir):
    if os.path.isdir(dest_dir):
        return

    if os.path.exists(zip_path):
        os.makedirs(dest_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(dest_dir)

def find_patient_root(root):
    for dirpath, dirnames, _ in os.walk(root):
        if any(d.startswith("patient") for d in dirnames):
            return dirpath
    return root

# * 解壓縮 (不使用外部 unzip 指令)
unzip_if_needed("training_image.zip", "../training_image")
unzip_if_needed("training_label.zip", "../training_label")

IMG_ROOT = find_patient_root("../training_image")
LBL_ROOT = find_patient_root("../training_label")

# * 建立輸出資料夾
for p in ["./datasets/train/images", "./datasets/train/labels",
          "./datasets/val/images", "./datasets/val/labels",
          "./datasets/test/images", "./datasets/test/labels"]:
    os.makedirs(p, exist_ok=True)

In [ ]:
#week10
def move_patients(start, end, split):
    # train/val 讀前幾筆病患，後面當作 test 保留完整影像
    moved = 0

    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient)
        if not os.path.isdir(img_dir):
            continue

        if split == "test":
            label_lookup = {}
            if os.path.isdir(lbl_dir):
                for frame in os.listdir(lbl_dir):
                    if frame.endswith(".txt"):
                        label_lookup[os.path.splitext(frame)[0]] = os.path.join(lbl_dir, frame)

            for frame in os.listdir(img_dir):
                if not frame.lower().endswith(".png"):
                    continue

                base = os.path.splitext(frame)[0]
                img_path = os.path.join(img_dir, frame)
                dst_img = f"./datasets/{split}/images/{base}.png"

                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)

                lbl_path = label_lookup.get(base)
                if lbl_path and os.path.exists(lbl_path):
                    dst_lbl = f"./datasets/{split}/labels/{base}.txt"
                    if os.path.exists(dst_lbl):
                        os.remove(dst_lbl)
                    shutil.move(lbl_path, dst_lbl)

                moved += 1

        else:
            if not os.path.isdir(lbl_dir):
                continue

            for lbl_name in os.listdir(lbl_dir):
                if not lbl_name.endswith(".txt"):
                    continue

                base = os.path.splitext(lbl_name)[0]
                img_path = os.path.join(img_dir, base + ".png")
                lbl_path = os.path.join(lbl_dir, lbl_name)
                if not os.path.exists(img_path):
                    continue

                dst_img = f"./datasets/{split}/images/{base}.png"
                dst_lbl = f"./datasets/{split}/labels/{base}.txt"

                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)

                if os.path.exists(dst_lbl):
                    os.remove(dst_lbl)
                shutil.move(lbl_path, dst_lbl)

                moved += 1

    return moved


In [ ]:
#week10
n_train = move_patients(1, 40, "train")
n_val = move_patients(41, 45, "val")
n_test = move_patients(46, 50, "test")

print("完成移動: train {} | val {} | test {}".format(n_train, n_val, n_test))

In [ ]:
print('訓練集圖片數量 : ',len(os.listdir("./datasets/train/images")))
print('訓練集標記數量 : ',len(os.listdir("./datasets/train/labels")))
print('驗證集圖片數量 : ',len(os.listdir("./datasets/val/images")))
print('驗證集標記數量 : ',len(os.listdir("./datasets/val/labels")))
print('測試集圖片數量 : ',len(os.listdir("./datasets/test/images")))
print('測試集標記數量 : ',len(os.listdir("./datasets/test/labels")))
print('總圖片數量: ',len(os.listdir("./datasets/train/images")) + len(os.listdir("./datasets/val/images")) + len(os.listdir("./datasets/test/images")))
print('總標記數量: ',len(os.listdir("./datasets/train/labels")) + len(os.listdir("./datasets/val/labels")) + len(os.listdir("./datasets/test/labels")))

訓練集圖片數量 :  2168
訓練集標記數量 :  2168
驗證集圖片數量 :  292
驗證集標記數量 :  292
測試集圖片數量 :  1817
測試集標記數量 :  327
總圖片數量:  4277
總標記數量:  2787


In [ ]:
from ultralytics import YOLO

# 1. 載入預訓練模型
model = YOLO('yolov8n.pt')  # 或是你原本使用的權重

# 2. 開始訓練 (這裡放入圖片 2 和 3 的改進參數)
results = model.train(
    data='/content/aortic_valve_colab.yaml',  # 你的 yaml 路徑
    epochs=20,                      # 訓練輪數
    imgsz=640,                       # 圖片大小

    # ==========================================
    # 對應圖片 2: 超參數微調 (Loss Weights)
    # ==========================================
    box=8.0,   # Box Loss 權重 (原本約 7.5，調大至 8)
    cls=0.4,   # Class Loss 權重 (原本 0.5，調小至 0.4)
    dfl=1.4,   # DFL Loss 權重 (原本 1.5，微調至 1.4)

    # 關於 Mosaic: YOLOv8 預設就會開啟 Mosaic
    # 並在最後 10 epoch 關閉 (close_mosaic=10 是預設值)，所以這點不用特別改，維持預設即可。

    # ==========================================
    # 對應圖片 3: 資料增強 (Data Augmentation)
    # 註：這裡設定的是 Online Augmentation，會在訓練時即時運算
    # ==========================================

    # 1. HSV 顏色增強 (圖片提到的 HSV-H/S/V)
    hsv_h=0.015,  # 色調 (預設 0.015)
    hsv_s=0.7,    # 飽和度 (預設 0.7)
    hsv_v=0.4,    # 亮度 (預設 0.4)

    # 2. 幾何變換 (圖片提到的 Offline Augmentation 轉換成訓練參數)
    fliplr=0.5,   # 水平翻轉 (Horizontal Flip): 50% 機率
    flipud=0.5,   # 垂直翻轉 (Vertical Flip): 50% 機率

    # 旋轉 90度 (Rotate 90)
    # YOLO 預設 degrees 是小角度旋轉(+/- 10度)。
    # 如果要模擬 "旋轉90度"，設大一點，例如 90 (代表 +/- 90度隨機)
    degrees=90,

    # 隨機縮放 (0.8 ~ 1.2)
    # scale 參數代表縮放增益。設為 0.2 代表 1.0 +/- 0.2 (即 0.8~1.2)
    scale=0.2,
)

Ultralytics 8.3.243 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.4, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/aortic_valve_colab.yaml, degrees=90, deterministic=True, device=None, dfl=1.4, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po

In [ ]:
#基礎訓練
# from ultralytics import YOLO
# model = YOLO('yolo11n.pt') #初次訓練使用YOLO官方的預訓練模型，如要使用自己的模型訓練可以將'yolo12n.pt'替換掉
# results = model.train(data="./aortic_valve_colab.yaml",
#             epochs=20, #跑幾個epoch
#             batch=16, #batch_size
#             imgsz=640, #圖片大小640*640
#             device=0 #使用GPU進行訓練
#             )

In [ ]:
import os
import shutil

# ================= 設定區 =================
base_root = "/content/datasets/test/images"  # 你的來源資料夾
dst_root1 = "/content/datasets/test/images1" # 目標資料夾 1
dst_root2 = "/content/datasets/test/images2" # 目標資料夾 2

# 建立輸出目錄
os.makedirs(dst_root1, exist_ok=True)
os.makedirs(dst_root2, exist_ok=True)

print(f"📂 正在搜尋圖片，來源：{base_root}")

# ================= 1. 萬能搜尋所有 PNG 檔案 =================
all_files = []

# os.walk 會把所有子目錄都翻一遍
for dirpath, dirnames, filenames in os.walk(base_root):
    for filename in filenames:
        # 只要是 png 結尾就抓起來 (不論是否在 patient 資料夾內)
        if filename.lower().endswith(".png"):
            full_path = os.path.join(dirpath, filename)
            all_files.append(full_path)

# 檢查是否有找到檔案
total_count = len(all_files)
if total_count == 0:
    print("❌ 錯誤：找不到任何 PNG 圖片！")
    print("   請檢查 base_root 路徑是否正確，或是資料夾裡面是否真的有圖片。")
    # 除錯用：印出該資料夾底下到底有什麼
    print(f"   目錄內容預覽: {os.listdir(base_root) if os.path.exists(base_root) else '目錄不存在'}")
else:
    # ================= 2. 排序與分配 =================
    all_files.sort()  # 排序確保每次跑結果一樣
    half = total_count // 2

    print(f"✅ 找到 {total_count} 張圖片，準備開始複製...")

    # 前半段複製到 images1
    for i, f in enumerate(all_files[:half]):
        try:
            shutil.copy(f, os.path.join(dst_root1, os.path.basename(f)))
        except Exception as e:
            print(f"   複製失敗: {f} -> {e}")

    # 後半段複製到 images2
    for i, f in enumerate(all_files[half:]):
        try:
            shutil.copy(f, os.path.join(dst_root2, os.path.basename(f)))
        except Exception as e:
             print(f"   複製失敗: {f} -> {e}")

    print(f"🎉 全部完成！")
    print(f"   - Images1: {len(os.listdir(dst_root1))} 張")
    print(f"   - Images2: {len(os.listdir(dst_root2))} 張")

📂 正在搜尋圖片，來源：/content/datasets/test/images
✅ 找到 1817 張圖片，準備開始複製...
🎉 全部完成！
   - Images1: 908 張
   - Images2: 909 張


In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train2/weights/best.pt') #請自行更改最新的best.pt檔路徑
results = model.predict(source="./datasets/test/images1/",
              save=True,
              imgsz=640,
              device=0
              )


image 1/908 /content/datasets/test/images1/patient0046_0001.png: 640x640 (no detections), 9.9ms
image 2/908 /content/datasets/test/images1/patient0046_0002.png: 640x640 (no detections), 7.9ms
image 3/908 /content/datasets/test/images1/patient0046_0003.png: 640x640 (no detections), 10.2ms
image 4/908 /content/datasets/test/images1/patient0046_0004.png: 640x640 (no detections), 11.6ms
image 5/908 /content/datasets/test/images1/patient0046_0005.png: 640x640 (no detections), 7.9ms
image 6/908 /content/datasets/test/images1/patient0046_0006.png: 640x640 (no detections), 8.2ms
image 7/908 /content/datasets/test/images1/patient0046_0007.png: 640x640 (no detections), 7.8ms
image 8/908 /content/datasets/test/images1/patient0046_0008.png: 640x640 (no detections), 8.2ms
image 9/908 /content/datasets/test/images1/patient0046_0009.png: 640x640 (no detections), 7.7ms
image 10/908 /content/datasets/test/images1/patient0046_0010.png: 640x640 (no detections), 7.8ms
image 11/908 /content/datasets/test/

In [ ]:
print(len(results))

908


In [ ]:
!mkdir ./predict_txt/
output_file = open('./predict_txt/images1.txt', 'w')
for i in range(len(results)):
    # 取得圖片檔名（不含副檔名）
    filename = results[i].path.split('/')[-1].split('.png')[0]

    # 取得預測框數量
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())

    # 如果有預測框
    if box_num > 0:
        for j in range(box_num):
            # 提取資訊
            label = int(boxes.cls[j].item())  # 類別
            conf = boxes.conf[j].item()       # 信心度
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()  # 邊界框座標

            # 建立一行資料
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)

# 關閉輸出檔案
output_file.close()

In [ ]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train2/weights/best.pt')
results = model.predict(source="./datasets/test/images2/",
              save=True,
              imgsz=640,
              device=0
              )


image 1/909 /content/datasets/test/images2/patient0048_0162.png: 640x640 (no detections), 31.9ms
image 2/909 /content/datasets/test/images2/patient0048_0163.png: 640x640 (no detections), 33.4ms
image 3/909 /content/datasets/test/images2/patient0048_0164.png: 640x640 (no detections), 36.0ms
image 4/909 /content/datasets/test/images2/patient0048_0165.png: 640x640 (no detections), 21.8ms
image 5/909 /content/datasets/test/images2/patient0048_0166.png: 640x640 (no detections), 19.3ms
image 6/909 /content/datasets/test/images2/patient0048_0167.png: 640x640 (no detections), 20.1ms
image 7/909 /content/datasets/test/images2/patient0048_0168.png: 640x640 (no detections), 18.3ms
image 8/909 /content/datasets/test/images2/patient0048_0169.png: 640x640 (no detections), 24.8ms
image 9/909 /content/datasets/test/images2/patient0048_0170.png: 640x640 (no detections), 29.1ms
image 10/909 /content/datasets/test/images2/patient0048_0171.png: 640x640 (no detections), 21.0ms
image 11/909 /content/datase

In [ ]:
output_file = open('./predict_txt/images2.txt', 'w')
for i in range(len(results)):
    # 取得圖片檔名（不含副檔名）
    filename = results[i].path.split('/')[-1].split('.png')[0]

    # 取得預測框數量
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())

    # 如果有預測框
    if box_num > 0:
        for j in range(box_num):
            # 提取資訊
            label = int(boxes.cls[j].item())  # 類別
            conf = boxes.conf[j].item()       # 信心度
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()  # 邊界框座標

            # 建立一行資料
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)

# 關閉輸出檔案
output_file.close()

In [ ]:
file1 = "./predict_txt/images1.txt"
file2 = "./predict_txt/images2.txt"
output = "./predict_txt/merged.txt"

with open(output, "w", encoding="utf-8") as fout:
    for f in [file1, file2]:
        if os.path.exists(f):
            with open(f, "r", encoding="utf-8") as fin:
                fout.writelines(fin.readlines())

print(f"合併完成 -> {output}")


合併完成 -> ./predict_txt/merged.txt


In [ ]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

NameError: name 'all_files' is not defined

In [ ]:
!yolo detect val model=/content/runs/detect/train2/weights/best.pt data=aortic_valve_colab.yaml split=test save_txt save_conf

Ultralytics 8.3.243 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2097.7±389.6 MB/s, size: 104.1 KB)
val: Scanning /content/datasets/test/labels... 327 images, 1490 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1817/1817 2.6Kit/s 0.7s
val: New cache created: /content/datasets/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 114/114 4.2it/s 27.5s
                   all       1817        327      0.746      0.717      0.768      0.312
Speed: 1.1ms preprocess, 3.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [ ]:
import os
import glob
import cv2
import shutil
import numpy as np

# ==================== 1. 設定區 (請確認這裡的路徑) ====================
# 你的測試圖片資料夾
TEST_IMAGES_DIR = 'datasets/test/images'
# 你的標準答案 (Ground Truth) 資料夾
TEST_LABELS_DIR = 'datasets/test/labels'

# ⚠️ 這裡填入你已經跑出來的資料夾名稱 (例如 'val', 'val2')
VAL_FOLDER_NAME = 'val'
# 這是 YOLO 預測結果的標籤路徑
PRED_DIR = f'/content/runs/detect/{VAL_FOLDER_NAME}/labels'

# 輸出視覺化圖片的資料夾
OUTPUT_DIR = 'final_report_output_visualized'

# ==================== 2. 顏色與工具函數定義 ====================
print(f"📂 正在讀取現有的預測結果：{PRED_DIR}")

if not os.path.exists(PRED_DIR):
    print(f"❌ 錯誤：找不到資料夾 {PRED_DIR}")
    print("   請檢查左側檔案欄，確認你的結果是在 'val', 'val2' 還是其他名稱？")
    raise SystemExit("找不到預測資料夾，請修改 VAL_FOLDER_NAME")

# 準備輸出目錄
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR)

# 顏色定義 (BGR 格式)
# 邏輯：綠色=標準答案, 黃色=預測正確, 紅色=預測錯誤
COLOR_GT = (0, 255, 0)      # 綠色 (標準答案 Ground Truth)
COLOR_TP = (0, 255, 255)    # 黃色 (預測正確 True Positive)
COLOR_FP = (0, 0, 255)      # 紅色 (誤報 False Positive)

stats = { "total_images": 0, "total_gt": 0, "total_pred": 0, "tp": 0, "fp": 0, "fn": 0, "dup": 0 }
report_rows = []

# 讀取圖片列表
img_paths = []
for root, dirs, files in os.walk(TEST_IMAGES_DIR):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_paths.append(os.path.join(root, file))
img_paths.sort()

if not img_paths:
    print(f"❌ 警告：在 {TEST_IMAGES_DIR} 找不到測試圖片！")

def parse_yolo_txt(path, width, height):
    """讀取 YOLO txt 轉為像素座標"""
    boxes = []
    if os.path.exists(path):
        with open(path, 'r') as f:
            for line in f:
                d = list(map(float, line.strip().split()))
                if len(d) < 5: continue
                cls = int(d[0])
                cx, cy, w, h = d[1], d[2], d[3], d[4]

                # 讀取信心度，如果是 GT 檔通常沒有這一欄，預設為 1.0
                conf = d[5] if len(d) > 5 else 1.0

                # 🔥 過濾低信心度的預測
                if len(d) > 5 and conf < 0.5:
                    continue

                x1 = int((cx - w/2) * width)
                y1 = int((cy - h/2) * height)
                x2 = int((cx + w/2) * width)
                y2 = int((cy + h/2) * height)
                boxes.append({'cls': cls, 'box': [x1, y1, x2, y2], 'conf': conf})
    return boxes

def compute_iou(boxA, boxB):
    """計算兩個框的重疊率 (IoU)"""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# ==================== 3. 開始比對與繪圖 ====================
print("📊 開始生成視覺化報告...")

for img_path in img_paths:
    stats["total_images"] += 1
    filename = os.path.basename(img_path)
    file_id = os.path.splitext(filename)[0]

    img = cv2.imread(img_path)
    if img is None: continue
    h, w = img.shape[:2]

    # 設定 Label 路徑
    gt_path = os.path.join(TEST_LABELS_DIR, file_id + '.txt')
    pred_path = os.path.join(PRED_DIR, file_id + '.txt')

    gts = parse_yolo_txt(gt_path, w, h)   # 標準答案
    preds = parse_yolo_txt(pred_path, w, h) # 預測結果

    stats["total_gt"] += len(gts)
    stats["total_pred"] += len(preds)

    gt_matched = [False] * len(gts)
    pred_matched = [False] * len(preds)

    # ---------------------------------------------------
    # 步驟 1: 找出 TP (正確預測)
    # 這裡會同時畫出「標準答案(綠)」和「預測結果(黃)」
    # ---------------------------------------------------
    for i, p in enumerate(preds):
        best_iou = 0
        best_gt_idx = -1

        # 尋找與這個預測框重疊最大的 GT
        for j, g in enumerate(gts):
            if p['cls'] == g['cls']:
                iou = compute_iou(p['box'], g['box'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j

        # 判定是否匹配成功 (IoU > 0.5)
        if best_iou >= 0.5:
            if not gt_matched[best_gt_idx]:
                gt_matched[best_gt_idx] = True
                pred_matched[i] = True
                stats["tp"] += 1

                # --- 核心修改區域 ---

                # 1. 畫出「標準答案 (GT)」 -> 綠色 (細線)
                # 即使抓對了，也畫出來讓你看差距
                g_box = gts[best_gt_idx]['box']
                cv2.rectangle(img, (g_box[0], g_box[1]), (g_box[2], g_box[3]), COLOR_GT, 1)

                # 2. 畫出「模型預測 (Pred)」 -> 黃色 (粗線)
                cv2.rectangle(img, (p['box'][0], p['box'][1]), (p['box'][2], p['box'][3]), COLOR_TP, 2)

                # 3. 標示信心度與 IoU
                text = f"{p['conf']:.2f} (IoU:{best_iou:.2f})"
                cv2.putText(img, text, (p['box'][0], p['box'][1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TP, 2)

            else:
                stats["dup"] += 1 # 重複偵測到同一個物體

    # ---------------------------------------------------
    # 步驟 2: 找出 FP (誤報) -> 畫紅色
    # ---------------------------------------------------
    for i, p in enumerate(preds):
        if not pred_matched[i]:
            stats["fp"] += 1
            report_rows.append(f"| {filename} | 誤報 (False Positive) | Conf: {p['conf']:.2f} |")
            cv2.rectangle(img, (p['box'][0], p['box'][1]), (p['box'][2], p['box'][3]), COLOR_FP, 2)
            cv2.putText(img, f"FP {p['conf']:.2f}", (p['box'][0], p['box'][1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_FP, 2)

    # ---------------------------------------------------
    # 步驟 3: 找出 FN (漏測) -> 畫綠色
    # 這些是沒有被抓到的標準答案，一樣要畫出來
    # ---------------------------------------------------
    for j, g in enumerate(gts):
        if not gt_matched[j]:
            stats["fn"] += 1
            report_rows.append(f"| {filename} | 漏測 (False Negative) | Class: {g['cls']} |")
            # 畫出標準答案 (粗線一點，強調這是漏掉的)
            cv2.rectangle(img, (g['box'][0], g['box'][1]), (g['box'][2], g['box'][3]), COLOR_GT, 2)
            cv2.putText(img, "FN (Miss)", (g['box'][0], g['box'][1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_GT, 2)

    # 儲存圖片
    cv2.imwrite(os.path.join(OUTPUT_DIR, filename), img)

# ==================== 4. 生成報告 ====================
report_path = '詳細驗證報告.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(f"# YOLO 模型驗證報告 (含標準答案比對)\n\n")
    f.write(f"## 1. 顏色圖例\n")
    f.write(f"- **<span style='color:green'>綠色框</span>**: 標準答案 (Ground Truth)。\n")
    f.write(f"  - 如果外面包著黃框：表示抓對了 (TP)。\n")
    f.write(f"  - 如果只有綠框寫著 FN：表示漏抓了 (FN)。\n")
    f.write(f"- **<span style='color:#CCCC00'>黃色框</span>**: 模型預測正確 (TP)，上方會有 IoU 數值。\n")
    f.write(f"- **<span style='color:red'>紅色框</span>**: 模型誤報 (FP)，也就是亂抓。\n\n")

    f.write(f"## 2. 統計數據\n")
    f.write(f"- 圖片總數: {stats['total_images']}\n")
    f.write(f"- GT 總數: {stats['total_gt']}\n")
    f.write(f"- 預測總數: {stats['total_pred']}\n\n")
    f.write(f"- **TP (準確)**: {stats['tp']}\n")
    f.write(f"- **FN (漏測)**: {stats['fn']}\n")
    f.write(f"- **FP (誤報)**: {stats['fp']}\n\n")

    f.write(f"## 3. 詳細異常列表 (前 200 筆)\n")
    f.write("| 檔名 | 錯誤類型 | 詳情 |\n")
    f.write("|---|---|---|\n")
    for row in report_rows[:200]:
        f.write(row + "\n")

print(f"\n✅ 全部完成！")
print(f"1. 視覺化結果已存入: {OUTPUT_DIR}/ (每張圖都有綠色標準答案)")
print(f"2. 報告已產生: {report_path}")

📂 正在讀取現有的預測結果：/content/runs/detect/val/labels
📊 開始生成視覺化報告...

✅ 全部完成！
1. 視覺化結果已存入: final_report_output_visualized/
2. 報告已產生: 詳細驗證報告.md
